Author:
        
        KIM, JeongYoong, jeongyoong@ccnets.org
        
    COPYRIGHT (c) 2024. CCNets. All Rights reserved.

# Heart Disease Dataset

Data Source: https://www.kaggle.com/datasets/mahdifaour/heart-disease-dataset/data

<a id="1"></a>
> <h1 style = 'font-family: Times New Roman'><b> <b style = 'color: #42c2f5'>1.</b> Import Necessary Libraries </b></h1>

In [1]:
import os
import sys
import warnings
warnings.filterwarnings("ignore")

path_append = "../../"
sys.path.append(path_append)  # Go up one directory from where you are.

In [2]:
import pandas  as pd
df = pd.read_csv(path_append + '../data/Heart Disease Dataset/Heart_Disease.csv')
df.head()

,sex,age,education,smokingStatus,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,CHDRisk
0,male,39,4,no,0,0,0,0,no,195,106.0,70.0,26.97,80,77,no
1,female,46,2,no,0,0,0,0,no,250,121.0,81.0,28.73,95,76,no
2,male,48,1,yes,20,0,0,0,no,245,127.5,80.0,25.34,75,70,no
3,female,61,3,yes,30,0,0,1,no,225,150.0,95.0,28.58,65,103,yes
4,female,46,3,yes,23,0,0,0,no,285,130.0,84.0,23.10,85,85,no


<a id="2"></a>
> <h1 style = 'font-family: Times New Roman'><b> <b style = 'color: #4290f5'>2.</b> Modeling: Preprocess </b></h1>

In [3]:
from tools.preprocessing.data_frame import auto_preprocess_dataframe
target_columns = ['CHDRisk']
df, description = auto_preprocess_dataframe(df, target_columns) 

Column 'diabetes' has 2 unique values.
Column 'sex' has 2 unique values.
Column 'smokingStatus' has 2 unique values.
Column 'CHDRisk' has 2 unique values.


,Min,Max,Mean,Std,Null Count,Scaled,Encoded
age,-5.808315,4.592238,-1.000000e+00,2.339339,0,Minmax,None
education,-0.982749,2.017251,1.138191e-16,1.022067,0,None,None
cigsPerDay,0.000000,3.500000,4.546687e-01,0.596618,0,Robust,None
BPMeds,-0.030120,0.969880,4.864066e-19,0.170942,0,None,None
prevalentStroke,-0.005750,0.994250,1.702423e-18,0.075623,0,None,None
prevalentHyp,-0.310789,0.689211,1.167376e-17,0.462880,0,None,None
totChol,-2.122807,6.421053,4.844257e-02,0.773303,0,Robust,None
sysBP,-1.679245,6.301887,1.652907e-01,0.831567,0,Robust,None
diaBP,-2.324786,4.136752,6.161710e-02,0.815829,0,Robust,None
BMI,-1.998987,6.361702,7.715471e-02,0.820912,0,Robust,None


In [4]:
import torch
from sklearn.model_selection import train_test_split
from tools.preprocessing.template_dataset import TemplateDataset

train_df, test_df = train_test_split(df, test_size=0.2, shuffle=False)
# predict the next value in the sequence
train_df_x = train_df.iloc[:, :-1] # all columns except the last one
train_df_y = train_df.iloc[:, -1:] # only the last column

test_df_x = test_df.iloc[:, :-1] # all columns except the last one
test_df_y = test_df.iloc[:, -1:] # only the last column

print('train df shape: ', train_df.shape)
print('test df shape: ', test_df.shape)
trainset = TemplateDataset(train_df_x, train_df_y)
testset = TemplateDataset(test_df_x, test_df_y)

train df shape:  (2921, 16)
test df shape:  (731, 16)


In [5]:
from tools.setting.data_config import DataConfig
from tools.setting.ml_params import MLParameters
from trainer_hub import TrainerHub

num_features = description['num_features']
num_classes = description['num_classes']
data_config = DataConfig(dataset_name = 'heart-disease', task_type='binary_classification', obs_shape=[num_features], label_size=num_classes)

#  Set training configuration from the AlgorithmConfig class, returning them as a Namespace object.
ml_params = MLParameters(ccnet_network = 'gpt', encoder_network = 'none')
ml_params.algorithm.error_function = 'mae'

# Set the device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

# Initialize the TrainerHub class with the training configuration, data configuration, device, and use_print and use_wandb flags
trainer_hub = TrainerHub(ml_params, data_config, device, use_print=True, use_wandb=False) 

Trainer Name: causal_trainer


ModelParameters Parameters:


,ccnet_config,ccnet_network,encoder_config,encoder_network
0,See details below,gpt,None,none



Detailed ccnet_config Configuration:


,ccnet_config_model_name,ccnet_config_num_layers,ccnet_config_d_model,ccnet_config_dropout,ccnet_config_obs_shape,ccnet_config_condition_dim,ccnet_config_z_dim
0,gpt,5,256,0.05,[15],2,6


TrainingParameters Parameters:


...

,clip_grad_range,decay_rate_100k,learning_rate,max_grad_norm,scheduler_type
0,None,0.05,0.0002,1.0,exponential


AlgorithmParameters Parameters:


,enable_diffusion,error_function,reset_pretrained
0,False,mae,False


DataConfig Parameters:


,dataset_name,task_type,obs_shape,label_size,explain_size,explain_layer,state_size,show_image_indices
0,heart-disease,binary_classification,[15],2,6,tanh,None,None


In [6]:
trainer_hub.train(trainset, testset)

Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Iterations:   0%|          | 0/45 [00:00<?, ?it/s]

Iterations:   0%|          | 0/45 [00:00<?, ?it/s]

Iterations:   0%|          | 0/45 [00:00<?, ?it/s]

[2/100][10/45][Time 10.68]
Unified LR across all optimizers: 0.0001993957766378747
--------------------Training Metrics--------------------
CCNet:  Three Gpt
Inf: 0.0542	Gen: 0.4495	Rec: 0.4435	E: 0.0602	R: 0.0483	P: 0.8388
--------------------Test Metrics------------------------
accuracy: 0.8125
precision: 0.4062
recall: 0.5000
f1_score: 0.4483



...

Iterations:   0%|          | 0/45 [00:00<?, ?it/s]

Iterations:   0%|          | 0/45 [00:00<?, ?it/s]

Iterations:   0%|          | 0/45 [00:00<?, ?it/s]

[91/100][5/45][Time 19.92]
Unified LR across all optimizers: 0.00017687838145997293
--------------------Training Metrics--------------------
CCNet:  Three Gpt
Inf: 0.0076	Gen: 0.1660	Rec: 0.1665	E: 0.0071	R: 0.0081	P: 0.3250
--------------------Test Metrics------------------------
accuracy: 0.7852
precision: 0.3926
recall: 0.5000
f1_score: 0.4398



Iterations:   0%|          | 0/45 [00:00<?, ?it/s]